In [29]:
!pip install -q pyomo

In [68]:
import pyomo.environ as pyomo
import random

# The Diet Problem

### **Problem**

Select foods that satisfy daily nutritional requirements at minimum cost. This problem can be formulated as a linear program, for which constraints limit the number of calories and the amount of vitamins, minerals, fats, sodium, and cholesterol in the diet.

In [42]:
infinity = float('inf')

model = AbstractModel()

#### Sets

In [43]:
# Foods
model.F = Set([])
# Nutrients
model.N = Set()

#### Parameters

In [44]:
# Cost of each food
model.c    = Param(model.F, {
    "Cheeseburger":     1.84,
    "Ham Sandwich":     2.19,
    "Hamburger":        1.84,
    "Fish Sandwich":    1.44,
    "Chicken Sandwich": 2.29,
    "Fries":            0.77,
    "Sausage Biscuit":  1.29, 
    "Lowfat Milk":      0.60,
    "Orange Juice":     0.72})
# Amount of nutrient in each food
model.a    = Param(model.F, model.N,{
    "Cheeseburger":     [510, 34, 28, 15,   6,    30,    20],
    "Ham Sandwich":     [370, 35, 24, 15,  10,    20,    20],
    "Hamburger":        [500, 42, 25,  6,   2,    25,    20],
    "Fish Sandwich":    [370, 38, 14,  2,   0,    15,    10],
    "Chicken Sandwich": [400, 42, 31,  8,  15,    15,     8],
    "Fries":            [220, 26,  3,  0,  15,     0,     2],
    "Sausage Biscuit":  [345, 27, 15,  4,   0,    20,    15], 
    "Lowfat Milk":      [110, 12,  9, 10,   4,    30,     0],
    "Orange Juice":     [ 80, 20,  1,  2, 120,     2,     2]})
# Lower and upper bound on each nutrient
model.Nmin = Param(model.N, {
    "Cal":      2000,
    "Carbo":     350,
    "Protein":    55,
    "VitA":      100,
    "VitC":      100,
    "Calc":      100,
    "Iron":      100})
model.Nmax = Param(model.N,{
    "Cal":       0,
    "Carbo":   375,
    "Protein":   0,
    "VitA":      0,
    "VitC":      0,
    "Calc":      0,
    "Iron":      0})
# Volume per serving of food
model.V    = Param(model.F, {
    "Cheeseburger":     4.0,
    "Ham Sandwich":     7.5,
    "Hamburger":        3.5,
    "Fish Sandwich":    5.0,
    "Chicken Sandwich": 7.3,
    "Fries":            2.6,
    "Sausage Biscuit":  4.1, 
    "Lowfat Milk":      8.0,
    "Orange Juice":     12.0})
# Maximum volume of food consumed
model.Vmax = Param({75.0})

#### Variables

In [45]:
# Number of servings consumed of each food
model.x = Var(model.F, within=NonNegativeIntegers)

#### Objective

In [46]:
# Minimize the cost of food that is consumed
def cost_rule(model):
    return sum(model.c[i]*model.x[i] for i in model.F)
model.cost = Objective(rule=cost_rule)

#### Constraints

In [47]:
# Limit nutrient consumption for each nutrient
def nutrient_rule(model, j):
    value = sum(model.a[i,j]*model.x[i] for i in model.F)
    return inequality(model.Nmin[j], value, model.Nmax[j])
model.nutrient_limit = Constraint(model.N, rule=nutrient_rule)

# Limit the volume of food consumed
def volume_rule(model):
    return sum(model.V[i]*model.x[i] for i in model.F) <= model.Vmax
model.volume = Constraint(rule=volume_rule)

### Solution

In [48]:
solver = SolverFactory('glpk')

In [ ]:
solution = solver.solve(model)

In [ ]:
display(model)

# Max Flow

### Problem

Find the maximum flow possible in a network from some given source node to a given sink node. Applications of the max flow problem include finding the maximum flow of orders through a job shop, the maximum flow of water through a storm sewer system, and the maximum flow of product through a product distribution system, among others.

In [50]:
from pyomo.environ import *

model = AbstractModel()

### Sets

In [55]:
# Nodes in the network
model.N = Set()
# Network arcs
model.A = Set(within=model.N*model.N)

In [57]:
nodes = ["Zoo", "A", "B", "C", "D", "E", "Home"]
arcs = [("Zoo","A"), ("Zoo","B"), ("A","C"), ("A","D"), ("B","A"), ("B","C"), ("C","D"), ("C","E"), ("D","E"), ("D","Home"), ("E","Home")]

### Parameters

In [61]:
# Source node
model.s = Param(model.N, {"Zoo"})
# Sink node
model.t = Param(model.N, {"Home"})
# Flow capacity limits
model.c = Param(model.A, {
    "Zoo": ["A", 11],
    "Zoo": ["B", 8],
    "A": ["C", 5],
    "A": ["D", 8],
    "B": ["A", 4],
    "B": ["C", 3],
    "C": ["D", 2],
    "C": ["E", 4],
    "D": ["E", 5],
    "D": ["Home", 8],
    "E": ["Home", 6],
})

### Variables

In [62]:
# The flow over each arc
model.f = Var(model.A, within=NonNegativeReals)

### Objective

In [63]:
# Maximize the flow into the sink nodes
def total_rule(model):
    return sum(model.f[i,j] for (i, j) in model.A if j == value(model.t))
model.total = Objective(rule=total_rule, sense=maximize)

### Constraints

In [64]:
# Enforce an upper limit on the flow across each arc
def limit_rule(model, i, j):
    return model.f[i,j] <= model.c[i, j]
model.limit = Constraint(model.A, rule=limit_rule)

# Enforce flow through each node
def flow_rule(model, k):
    if k == value(model.s) or k == value(model.t):
        return Constraint.Skip
    inFlow  = sum(model.f[i,j] for (i,j) in model.A if j == k)
    outFlow = sum(model.f[i,j] for (i,j) in model.A if i == k)
    return inFlow == outFlow
model.flow = Constraint(model.N, rule=flow_rule)

In [69]:
solver = pyomo.SolverFactory('glpk')

In [ ]:
solution = solver.solve(model)

In [ ]:
display(model)